# Exercise 4

In [36]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import copy

import numpy as np
import unidecode
import string
import random
import time
import math

# Tensorboard for visualizing
from torch.utils.tensorboard import SummaryWriter

### Import Tiny Shakespeare

### Model from https://github.com/spro/char-rnn.pytorch/blob/master/model.py

In [14]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, model="gru", n_layers=1):
        super(CharRNN, self).__init__()
        self.model = model.lower()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        if self.model == "gru":
            self.rnn = nn.GRU(hidden_size, hidden_size, n_layers)
        elif self.model == "lstm":
            self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        batch_size = input.size(0)
        encoded = self.encoder(input)
        output, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        output = self.decoder(output.view(batch_size, -1))
        return output, hidden

    def forward2(self, input, hidden):
        encoded = self.encoder(input.view(1, -1))
        output, hidden = self.rnn(encoded.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self, batch_size):
        if self.model == "lstm":
            return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
                    Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))


### helpers.py from https://github.com/spro/char-rnn.pytorch/blob/master/helpers.py

In [15]:
# https://github.com/spro/char-rnn.pytorch

# Reading and un-unicode-encoding data

all_characters = string.printable
n_characters = len(all_characters)

def read_file(filename):
    file = unidecode.unidecode(open(filename).read())
    return file, len(file)

# Turning a string into a tensor

def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = all_characters.index(string[c])
        except:
            continue
    return tensor

# Readable time elapsed

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

### generate.py from https://github.com/spro/char-rnn.pytorch/blob/master/generate.py

In [29]:
#!/usr/bin/env python
# https://github.com/spro/char-rnn.pytorch

import torch
import os
import argparse

def generate(decoder, prime_str='A', predict_len=100, temperature=0.8, cuda=True):
    hidden = decoder.init_hidden(1)
    prime_input = Variable(char_tensor(prime_str).unsqueeze(0))

    if cuda:
        hidden = hidden.cuda()
        prime_input = prime_input.cuda()
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[:,p], hidden)

    inp = prime_input[:,-1]

    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)

        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = Variable(char_tensor(predicted_char).unsqueeze(0))
        if cuda:
            inp = inp.cuda()

    return predicted

# Run as standalone script
#if __name__ == '__main__':

# Parse command line arguments
#     argparser = argparse.ArgumentParser()
#     argparser.add_argument('filename', type=str)
#     argparser.add_argument('-p', '--prime_str', type=str, default='A')
#     argparser.add_argument('-l', '--predict_len', type=int, default=100)
#     argparser.add_argument('-t', '--temperature', type=float, default=0.8)
#     argparser.add_argument('--cuda', action='store_true')
#     args = argparser.parse_args()

filename = "tinyShakespear.pt"
start = "The"

decoder = torch.load(filename)
del filename
print(generate(decoder, prime_str=start))

The hear me here all him.

CORIO:
How fares been, sir, that he at this pition,
No fortune, as the times


### train.py from https://github.com/spro/char-rnn.pytorch/blob/master/train.py

In [ ]:
#!/usr/bin/env python
# https://github.com/spro/char-rnn.pytorch

import torch
import torch.nn as nn
from torch.autograd import Variable
import argparse
import os

from tqdm import tqdm


# Parse command line arguments
# argparser = argparse.ArgumentParser()
# argparser.add_argument('filename', type=str, default='tinyShakespear.txt')
# argparser.add_argument('--model', type=str, default="gru")
# argparser.add_argument('--n_epochs', type=int, default=2000)
# argparser.add_argument('--print_every', type=int, default=100)
# argparser.add_argument('--hidden_size', type=int, default=100)
# argparser.add_argument('--n_layers', type=int, default=2)
# argparser.add_argument('--learning_rate', type=float, default=0.01)
# argparser.add_argument('--chunk_len', type=int, default=200)
# argparser.add_argument('--batch_size', type=int, default=100)
# argparser.add_argument('--shuffle', action='store_true')
# argparser.add_argument('--cuda', action='store_true')
# args = argparser.parse_args()

arg_filename = "tinyShakespear.txt"
arg_model = "gru"
arg_n_epochs = 2000
arg_print_every = 2000
arg_hidden_size = 100
arg_n_layers = 2
arg_learning_rate = 0.01
arg_chunk_len = 200
arg_batch_size = 100
arg_shuffle = True
arg_cuda = True


if arg_cuda:
    print("Using CUDA")

file, file_len = read_file(arg_filename)

def make_vocab(input_seq, input_len):
    my_vocab = []
    my_word = ""
    for i in range(input_len):
        my_char = input_seq[i]
        if my_char == ' ' or my_char == '\n' or my_char == '\0':
            if my_word.len() > 0:
                my_vocab.append(my_word)
            my_word = ""
        else:
            my_word = my_word + my_char;
    return my_vocab

vocab = make_vocab(read_file(arg_filename))

def random_training_set(chunk_len, batch_size):
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for bi in range(batch_size):
        start_index = random.randint(0, file_len - chunk_len)
        end_index = start_index + chunk_len + 1
        chunk = file[start_index:end_index]
        set_vocab = make_vocab(chunk, chunk.len())
        #for each element in set_vocab, match to index of same word in vocab
        inp[bi] = char_tensor(chunk[:-1])
        target[bi] = char_tensor(chunk[1:])
    inp = Variable(inp)
    target = Variable(target)
    if arg_cuda:
        inp = inp.cuda()
        target = target.cuda()
    return inp, target

def train(inp, target):
    hidden = decoder.init_hidden(arg_batch_size)
    if arg_cuda:
        hidden = hidden.cuda()
    decoder.zero_grad()
    loss = 0

    for c in range(arg_chunk_len):
        output, hidden = decoder(inp[:,c], hidden)
        loss += criterion(output.view(arg_batch_size, -1), target[:,c])

    loss.backward()
    decoder_optimizer.step()

    return loss.item()/ arg_chunk_len

def save():
    save_filename = os.path.splitext(os.path.basename(arg_filename))[0] + '.pt'
    torch.save(decoder, save_filename)
    print('Saved as %s' % save_filename)

# Initialize models and start training

decoder = CharRNN(
    n_characters,
    arg_hidden_size,
    n_characters,
    model = arg_model,
    n_layers = arg_n_layers
)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=arg_learning_rate)
criterion = nn.CrossEntropyLoss()

if arg_cuda:
    decoder.cuda()

start = time.time()
all_losses = []
loss_avg = 0

writer = SummaryWriter()

try:
    print("Training for %d epochs..." % arg_n_epochs)
    for epoch in tqdm(range(1, arg_n_epochs + 1)):
        loss = train(*random_training_set(arg_chunk_len, arg_batch_size))
        writer.add_scalar("Perplexity", np.exp(loss), epoch)
        loss_avg += loss

        if epoch % arg_print_every == 0:
            print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / arg_n_epochs * 100, loss))
            print(generate(decoder, 'Wh', 100, cuda=arg_cuda), '\n')

    print("Saving...")
    save()

except KeyboardInterrupt:
    print("Saving before quit...")
    save()

writer.flush()

Using CUDA
Training for 2000 epochs...


 30%|███       | 606/2000 [03:58<09:27,  2.46it/s]

In [32]:
 def simple_tokenizer(text):
    '''
    Takes a text and returns a list of the individual words
    Args:
        text (str): The text to tokenize
    Returns ([str]): A list of individual words
    '''
    ## WRITE A TOKENIZER (Hint: use the 'split' function)
    return text.split()

class SimpleVocabulary():

    def __init__(self, corpus):
        '''
        Takes a corpus, tokenizes it, and sets the internal variables needed
        to return the index of any given word (and what word has a given index)
        Args:
            corpus ([str]): The corpus as a list of non-tokenized texts
        '''
        ## WRITE A FUNCTION TO INITIALIZE THE VOCABULARY
        # HINT: Create a dictionary that contains the index for each unique word and antother to contain the word for a given index
        # HINT: Each new, previously unseen token from the tokenized corpus, should be entered into the dictionaries

        current_index = 0
        self.token2index = {}
        self.index2token = []
        for text in corpus:
            for word in simple_tokenizer(text):
                if word not in self.token2index:
                    self.index2token.append(word)
                    self.token2index.update({word : current_index})
                    current_index += 1


    def get_index(self, token):
        '''
        Takes a token and returns the index of that token
        Args:
            token (str): A token
        Returns (int): The index of the given token
        '''
        ## WRITE A FUNCTION THAT RETURNS THE INDEX OF A GIVEN TOKEN
        return self.token2index.get(token)


    def get_token(self, index):
        '''
        Takes an index and returns the token that index represents
        Args:
            index (int): An index
        Returns (str): The token that the given index represents
        '''
        ## WRITE A FUNCTION THAT RETURNS THE TOKEN OF A GIVEN INDEX
        return self.index2token[index]


    def __len__(self):
        '''
        Returns (int): The number of words in the vocabulary
        '''
        ## WRITE A FUNCTION THAT RETURNS THE LENGTH OF THE VOCABULARY
        return len(self.token2index)

def index_sequence(sequence, vocabulary):
    '''
    Takes a tokenized text and a vocabulary and returns an indexed sequence
    Args:
        sequence ([str]): A list of individual tokens
    Returns ([int]): A list of indexes
    '''

    return [vocabulary.get_index(token) for token in sequence]
    

2eF 3hom at your privilued, nor
Some company, our husband of not to pursue.

RICHMOND:
No loddle of a fire



fdghject you to twell'd, sir.

Nurse:
The shoppited conference, uncle, stay, but no,
Which art thou bed, 



AbCd3 GOZUMWARD:
So shall I prepore, and alive me up.
3 KING HENRY VI

PROSPERO:
Before; thyself at the p


The heart not the present Mowbray
But the clouded the regree and men
They shall chief thoughs so soul t



What is here?

CATESBY:
Why, blush more action, loyal know my morning
time. Let me not, holy hand in the ch



Shall I give the bright his blood,
Than be up the heavens, the counsel of the sile
for the marters combard hath 



X087hNYB BHN BYFVuhsdbsoe: full
The spirit death, for the faker!

LUCIO:
Mistressly soul will the cover for that put
Finlma



